In [18]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

img_width, img_height = 224, 224
batch_size = 32

classs_names = ['book', 'box', 'case', 'drops', 'flash', 'glass', 'magazine', 'pen']

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


for layer in base_model.layers:
    layer.trainable = False

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'data',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
    'data',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True)

checkpoint = callbacks.ModelCheckpoint(
    'best_model.h5', 
    save_best_only=True, 
    monitor='val_accuracy', 
    mode='max')

history = model.fit(
        train_generator,
        epochs=40,
        validation_data=valid_generator,
        callbacks=[early_stopping, checkpoint]
)

val_acc = history.history['val_accuracy'][-1]
print('Validation Accuracy:', val_acc)

history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot()

print('Minimum val_loss', history_df['val_loss'].min()) 

Found 106 images belonging to 8 classes.
Found 106 images belonging to 8 classes.


Epoch 1/40
2/4 [==============>...............] - ETA: 6s - loss: 4.2880 - accuracy: 0.1250 

KeyboardInterrupt: 

In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Загружаем модель
model = load_model('best_model.h5')

# Загружаем изображение для предсказания
img_path = 'pred/image3.jpg'
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

# Получаем предсказание модели
preds = model.predict(x)
predicted_class = np.argmax(preds)

print('Predicted class:', predicted_class)

1/1 [==============================] - 0s 213ms/step
Predicted class: 5
